In [8]:
%pip install requests
%pip install rsa


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import requests
import rsa
import json
import binascii
import json
import random

# G_BaseURL = "http://192.168.80.147/TESTwebservices_SWC"
G_BaseURL = "http://78.158.168.230/TESTwebservices_SWC"
# G_BaseURL = "http://192.168.80.134/Rahkaran"
G_UserName = "admin"
G_PassWord = "admin"

def GiveMePostManBody(json):
    #json = json.replace('"', '\\"')
    #json = json.replace("'", '"')
    print(json)




def hex_string_to_bytes(hex_string):
    return binascii.unhexlify(hex_string)
def bytes_to_hex_string(byte_array):
    return binascii.hexlify(byte_array).decode()
def login(user_name=G_UserName, password=G_PassWord):
    url = G_BaseURL + "/Services/Framework/AuthenticationService.svc"
    session_url = url + "/session"
    login_url = url + "/login"
    response = requests.get(session_url)
    if response.status_code != 200:
        raise Exception("GET /session {}".format(response.status_code))
    session = json.loads(response.text)
    m = hex_string_to_bytes(session["rsa"]["M"])
    e = hex_string_to_bytes(session["rsa"]["E"])
    rsa_key = rsa.PublicKey(
        int.from_bytes(m, byteorder="big"), int.from_bytes(e, byteorder="big")
    )
    session_plus_password = session["id"] + "**" + password
    encrypted_password = rsa.encrypt(session_plus_password.encode(), rsa_key)
    headers = {"content-Type": "application/json"}
    data = {
        "sessionId": session["id"],
        "username": user_name,
        "password": bytes_to_hex_string(encrypted_password),
    }
    response = requests.post(login_url, headers=headers, data=json.dumps(data))
    if response.status_code != 200:
        raise Exception("POST /login {}".format(response.status_code))
    if response.headers["Set-Cookie"].split(",")[2].split(";")[0].strip()[:2] =="sg":
        session = response.headers["Set-Cookie"].split(",")[2].split(";")[0].strip()
    elif response.headers["Set-Cookie"].split(",")[1].split(";")[0].strip()[:2] == "sg":
        session = response.headers["Set-Cookie"].split(",")[1].split(";")[0].strip()
    return session


session = login()
print(session)

sg-auth-TESTwebservices_SWC=3303f891-49b8-44e3-9330-5fd660df03ef


In [19]:
"""
    get all  GetRegionalDivisionList
"""



def GetRegionalDivisionList(session):
    url = (
        G_BaseURL
        + "/General/AddressManagement/Services/AddressManagementWebService.svc/GetRegionalDivisionList"
    )
    headers = {"Cookie": session, "content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception("GET /GetAllCostCenters {}".format(response.status_code))
    return response.json()


# Use the session_id and auth_cookie from the login function
# cost_centers = get_all_cost_centers(session)
# print(cost_centers)
# json_str = json.dumps(your_list)
# parsed = json.loads(cost_centers)

# Pretty print the parsed data
# print(json.dumps(parsed, indent=4))

print(
    json.dumps(
        json.loads(json.dumps(GetRegionalDivisionList(session))),
        indent=4,
        ensure_ascii=False,
    )
)
# """

[
    {
        "Code": null,
        "Name": "کشور ایران",
        "ParentID": null,
        "RegionalDivisionID": 115,
        "Type": 1
    },
    {
        "Code": null,
        "Name": "استان آذربایجان شرقی",
        "ParentID": 115,
        "RegionalDivisionID": 210,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان آذربایجان غربی",
        "ParentID": 115,
        "RegionalDivisionID": 211,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان اصفهان",
        "ParentID": 115,
        "RegionalDivisionID": 212,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان خراسان رضوی",
        "ParentID": 115,
        "RegionalDivisionID": 213,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان خوزستان",
        "ParentID": 115,
        "RegionalDivisionID": 214,
        "Type": 2
    },
    {
        "Code": null,
        "Name": "استان فارس",
        "ParentID": 115,
        "RegionalDivisio

In [33]:
# """


def salesStatementRegister(session, data):
    url = G_BaseURL + "/System/BusinessRuleEngine/Service.svc/call"
    headers = {"Cookie": session, "content-Type": "application/json"}
    JsonData = {"Name": "SalesStatementRegister", "Parameters": json.dumps(data)}
    response = requests.post(url, headers=headers, data=json.dumps(JsonData))
    if response.status_code != 200:
        #raise Exception("salesStatementRegister".format(response.status_code))
        print("hi")
    return response.json()


SalesStatement = {
    "Number": "",
    "SupplierCode": "43982",
    "Date": "05/12/2024 00:00:00",
    "SalesOfficeRef": "1",  # همیشه 1
    "SalesAreaRef": "1",  # همیشه 1
    "RecipientType": "4",  # نوع تحویل گیرنده
    "RecipientRef": 88,  # شناسه مشتری تحویل گیرنده یا موسسه حمل
    "SellerBrokerRef": "1",  # همیشه 1
    "DeliveryAddressType": "1",  # نوع نشانی همیشه مشتری  1 موسسه حمل 2
    "StatementType": "1",  # همیشه 1 نوع اعلامیه
    "DeliverySpace": "08/09/2028 00:00:00",
    "ContractNo": str(random.randint(1, 99999999)),  # شماره قرارداد
    "PlantRef": "1",  # شماره مرکز همیشه 1
    "ReferenceType": "1",  # ؟همیشه 1
    "SalesTypeRef": "1",  # نوع فروش همیشه 1
    "InventoryRef": "3",  # شناسه انبار همیشه 3
    "SalesStatementWageResults": [  # کارمزدها
        {
            "BrokerServiceRef": "1",  # کارمزد فروش بورسی
            "CalculationPrecedence": "0",  # همیشه 0
            "CurrencyRef": "1",  # همیشه 1 نوع ارز
            "Value": "2502",  # مبلغ کارمز
            "ValueSetByUser": True,  # همیشه درست
        },
        {
            "BrokerServiceRef": "2",  # کارمزد حق درج
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "300",
            "ValueSetByUser": True,
        },
    ],
    "SalesStatementItems": [  # اقلام
        {
            "ProductRef": "9216",  # شناسه محصول همیشه 9216؟
            "ProductUnitRef": "1",  # واحد سنجش همیشه 1
            "InitialQuantity": "11",  # مقدار
            "Fee": "900000",  # نرخ
        }
    ],
}
# data = {"SalesStatement": SalesStatement}
# ERA = salesStatementRegister(session, {"SalesStatement": SalesStatement})
print(salesStatementRegister(session, {"SalesStatement": SalesStatement}))

{"SalesStatementID":205,"Errore":""}


In [26]:
# """


def salesStatementRegister(session, data):
    url = G_BaseURL + "/System/BusinessRuleEngine/Service.svc/call"
    headers = {"Cookie": session, "content-Type": "application/json"}
    JsonData = {"Name": "SalesStatementRegister", "Parameters": json.dumps(data)}
    response = requests.post(url, headers=headers, data=json.dumps(JsonData))
    #print(json.dumps(JsonData))
    return response.json(),json.dumps(JsonData)

SalesStatement = {
    "Date": "2024-07-02T11:10:25.387",
    "SalesOfficeRef": 1,
    "RecipientRef": 73,
    "RecipientType": 4,
    "SellerBrokerRef": 1,
    "DeliveryAddressType": 1,
    "StatementType": 1,
    "SalesAreaRef": 1,
    "DeliveryAddressRef": 0,
    "ReferenceType": 1,
    "SupplierCode": "43982",
    "DeliverySpace": "2024-07-06T00:00:00",
    "ContractNo": str(random.randint(1, 99999999)),
    "ContractDate": "2024-07-01T00:00:00",
    "PlantRef": 1,
    "InventoryRef": 3,
    
    "DeliveryPlace" : "تهران",
    "Producer" : "2316",
    "OperationalCurrencyExchangerAte":1,
    "SalesStatementWageResults": [
        {
            "BrokerServiceRef": "1",  # کارمزد فروش بورسی
            "CalculationPrecedence": "0",  # همیشه 0
            "CurrencyRef": "1",  # همیشه 1 نوع ارز
            "Value": "2502",  # مبلغ کارمز
            "ValueSetByUser": True,  # همیشه درست
        },
        {
            "BrokerServiceRef": "2",  # کارمزد حق درج
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "300",
            "ValueSetByUser": True,
        },
    ],
    "SalesTypeRef": 1,
    "SalesStatementItems": [
        {
            "ProductRef": 9216,
            "ProductUnitRef": 1,
            "InitialQuantity": 25000.00000,
            "Fee": 35502.00000,
            "Price": 887550000.00000,
            
        }
    ],
}

data = {"SalesStatement": SalesStatement}
salesStatementResults = salesStatementRegister(session, {"SalesStatement": SalesStatement})
print(salesStatementResults[0])
print(salesStatementResults[1])
#print(salesStatementResults)

{"SalesStatementID":3540,"Errore":"","SalesStatementNumber":""}
{"Name": "SalesStatementRegister", "Parameters": "{\"SalesStatement\": {\"Date\": \"2024-07-02T11:10:25.387\", \"SalesOfficeRef\": 1, \"RecipientRef\": 73, \"RecipientType\": 4, \"SellerBrokerRef\": 1, \"DeliveryAddressType\": 1, \"StatementType\": 1, \"SalesAreaRef\": 1, \"DeliveryAddressRef\": 0, \"ReferenceType\": 1, \"SupplierCode\": \"43982\", \"DeliverySpace\": \"2024-07-06T00:00:00\", \"ContractNo\": \"24997093\", \"ContractDate\": \"2024-07-01T00:00:00\", \"PlantRef\": 1, \"InventoryRef\": 3, \"DeliveryPlace\": \"\\u062a\\u0647\\u0631\\u0627\\u0646\", \"Producer\": \"2316\", \"OperationalCurrencyExchangerAte\": 1, \"SalesStatementWageResults\": [{\"BrokerServiceRef\": \"1\", \"CalculationPrecedence\": \"0\", \"CurrencyRef\": \"1\", \"Value\": \"2502\", \"ValueSetByUser\": true}, {\"BrokerServiceRef\": \"2\", \"CalculationPrecedence\": \"0\", \"CurrencyRef\": \"1\", \"Value\": \"300\", \"ValueSetByUser\": true}], \"

In [1]:
import json

# Step 1: Create the nested SalesStatement dictionary
sales_statement = {
    "Date": "2024-07-02T11:10:25.387",
    "SalesOfficeRef": 1,
    "RecipientRef": 73,
    "RecipientType": 4,
    "SellerBrokerRef": 1,
    "DeliveryAddressType": 1,
    "StatementType": 1,
    "SalesAreaRef": 1,
    "DeliveryAddressRef": 0,
    "ReferenceType": 1,
    "SupplierCode": "43982",
    "DeliverySpace": "2024-07-06T00:00:00",
    "ContractNo": "7138080",
    "ContractDate": "2024-07-01T00:00:00",
    "PlantRef": 1,
    "InventoryRef": 3,
    "DeliveryPlace": "تهران",  # Example of a non-ASCII string
    "Producer": "2316",
    "OperationalCurrencyExchangerAte": 1,
    "SalesStatementWageResults": [
        {
            "BrokerServiceRef": "1",
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "2502",
            "ValueSetByUser": True,
        },
        {
            "BrokerServiceRef": "2",
            "CalculationPrecedence": "0",
            "CurrencyRef": "1",
            "Value": "300",
            "ValueSetByUser": True,
        },
    ],
    "SalesTypeRef": 1,
    "SalesStatementItems": [
        {
            "ProductRef": 9216,
            "ProductUnitRef": 1,
            "InitialQuantity": 25000.0,
            "Fee": 35502.0,
            "Price": 887550000.0,
        }
    ],
}

# Step 2: Convert the SalesStatement dictionary to a JSON string
sales_statement_json = json.dumps(sales_statement)

# Step 3: Create the main dictionary
main_data = {"Name": "SalesStatementRegister", "Parameters": sales_statement_json}

# Step 4: Convert the main dictionary to a JSON string
final_json = json.dumps(main_data)

# Print the final JSON string
print(final_json)

{"Name": "SalesStatementRegister", "Parameters": "{\"Date\": \"2024-07-02T11:10:25.387\", \"SalesOfficeRef\": 1, \"RecipientRef\": 73, \"RecipientType\": 4, \"SellerBrokerRef\": 1, \"DeliveryAddressType\": 1, \"StatementType\": 1, \"SalesAreaRef\": 1, \"DeliveryAddressRef\": 0, \"ReferenceType\": 1, \"SupplierCode\": \"43982\", \"DeliverySpace\": \"2024-07-06T00:00:00\", \"ContractNo\": \"7138080\", \"ContractDate\": \"2024-07-01T00:00:00\", \"PlantRef\": 1, \"InventoryRef\": 3, \"DeliveryPlace\": \"\\u062a\\u0647\\u0631\\u0627\\u0646\", \"Producer\": \"2316\", \"OperationalCurrencyExchangerAte\": 1, \"SalesStatementWageResults\": [{\"BrokerServiceRef\": \"1\", \"CalculationPrecedence\": \"0\", \"CurrencyRef\": \"1\", \"Value\": \"2502\", \"ValueSetByUser\": true}, {\"BrokerServiceRef\": \"2\", \"CalculationPrecedence\": \"0\", \"CurrencyRef\": \"1\", \"Value\": \"300\", \"ValueSetByUser\": true}], \"SalesTypeRef\": 1, \"SalesStatementItems\": [{\"ProductRef\": 9216, \"ProductUnitRef\"